Зберемо відео з variable frame rate з "плейлістом" для демуксера вказаним нижче. Усього вісім фреймів з тривалістю 1.25 секунд плюс заключний фрейм без тривалості.

In [1]:
! cat concat.txt

file frames/black_frame.jpg
duration 1.25
file frames/white_frame.jpg
duration 1.25
file frames/black_frame.jpg
duration 1.25
file frames/white_frame.jpg
duration 1.25
file frames/black_frame.jpg
duration 1.25
file frames/white_frame.jpg
duration 1.25
file frames/black_frame.jpg
duration 1.25
file frames/white_frame.jpg
duration 1.25
file frames/white_frame.jpg


Впевнимося що сума тривалостей в полі `duration` дорівнює десяти.

In [2]:
! cat concat_count.sh

#!/bin/bash

set -eux

grep duration concat.txt | awk '{ sum += $2 } END { print sum }'


In [3]:
! bash ./concat_count.sh

+ grep duration concat.txt
+ awk '{ sum += $2 } END { print sum }'
10


Зберемо відео з плейліста вказаши що потрібен variable frame rate та перевіримо його тривалість з `ffprobe`.

In [4]:
! cat ./ffconcat.sh

#!/bin/bash

set -eux

ffmpeg -hide_banner -f concat -i concat.txt -fps_mode vfr -pix_fmt yuv420p -c:v libx264 output.mp4 -y &>/dev/null
ffprobe -hide_banner -show_streams output.mp4 | grep --color "duration="


In [5]:
! bash ./ffconcat.sh

+ ffmpeg -hide_banner -f concat -i concat.txt -fps_mode vfr -pix_fmt yuv420p -c:v libx264 output.mp4 -y
+ ffprobe -hide_banner -show_streams output.mp4
+ grep --color duration=
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'output.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf61.1.100
  Duration: 00:00:10.04, start: 0.000000, bitrate: 58 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 1920x1080 [SAR 1:1 DAR 16:9], 57 kb/s, 0.89 fps, 25 tbr, 12800 tbn (default)
      Metadata:
        handler_name    : VideoHandler
        vendor_id       : [0][0][0][0]
        encoder         : Lavc61.3.100 libx264
duration=10.040000


Розберемо відео на фрейми, написавши в оверлей таймстампу фрейма. Це можна зробити параметром `pts` у фільтрі `drawtext`.

In [6]:
! cat extract.sh

#!/bin/bash

set -eux

rm -frd ./tmp
mkdir -p ./tmp
ffmpeg -i output.mp4 -vf "drawtext=fontfile=arialbd.ttf:fontsize=24:fontcolor=white:box=1:boxcolor=black@0.5:boxborderw=5:x=(w-tw)/2:y=h-th-10:text='%{pts\:hms}'" -vsync 0 ./tmp/output_frame_%d.png


In [7]:
! bash ./extract.sh &> /dev/null

Таймстампи мають розбіжності з "плейлістом". Так, другий фрейм мав би починатись на 1.26 секунді, натомість він починається 1.24

![image](./tmp/output_frame_2.png)

Висновок: `ffmpeg` працює неочікувано, таймстампи не відповідають заданим тривалостям.